In [15]:
import pandas as pd
import numpy as np 
import zipfile
from scipy import stats

In [16]:
epn= pd.read_csv("results/EPN_all_data.tsv",sep="\t")

In [17]:
epn[epn["Kids_First_Biospecimen_ID_RNA"].notna()]

In [26]:
epn[epn["Kids_First_Biospecimen_ID_RNA"].isna()].index

Int64Index([23, 35, 48, 65], dtype='int64')